This analysis contains information about 79 customers


 ------------------------------  Summary of License Costs  ------------------------------
                              Product Name License type  Quantity Contract ARR Average Discount
                         Active Data Guard          NUP       225       $9,753             -81%
                         Active Data Guard    Processor       182     $155,707             -93%
                      Advanced Compression          NUP       175       $3,020             -92%
                      Advanced Compression    Processor       231     $225,919             -91%
                         Advanced Security    Processor       277     $323,771             -92%
Data Masking Pack for Non Oracle Databases    Processor         5       $6,461             -89%
                        Database In-Memory    Processor         5       $8,008             -93%
        Database Lifecycle Management Pack    Processor        17      $16,121        